In [1]:
import cupy as cp
import numpy as np
import zarr
import h5py
from decorrelation.shp import ks_test
from decorrelation.co import emperical_co,emperical_co_sp, regularize_spectral, wherePD

## rslc

In [2]:
rslc = cp.load('./rslc.npy')

In [3]:
zarr.save_array('rslc.zarr', cp.asnumpy(rslc),chunks=(1000,1000),compressor=None)

## SHP statistics and p-value

In [4]:
az_half_win = 5
r_half_win = 5
az_win = 2*az_half_win+1
r_win = 2*r_half_win+1

In [5]:
rmli = cp.abs(rslc)**2
sorted_rmli = cp.sort(rmli,axis=-1)
del rmli
dist,p = ks_test(sorted_rmli,az_half_win=az_half_win,r_half_win=r_half_win)

In [6]:
zarr.save_array('dist_ks.zarr', cp.asnumpy(dist),chunks=(1000,1000,az_win,r_win),compressor=None)
zarr.save_array('p_ks.zarr', cp.asnumpy(p),chunks=(1000,1000,az_win,r_win),compressor=None)

In [7]:
del dist

## selected SHP 

In [20]:
is_shp = (p < 0.05) & (p >= 0.0)
del p

In [21]:
zarr.save_array('is_shp.zarr', cp.asnumpy(is_shp),chunks=(1000,1000,az_win,r_win),compressor=None)

## DS

In [23]:
shp_num = cp.count_nonzero(is_shp,axis=(-2,-1))
is_ds = shp_num >= 50

In [24]:
zarr.save_array('is_ds.zarr', cp.asnumpy(is_ds),chunks=(1000,1000),compressor=None)

## covariance, coherence

In [25]:
is_shp_ds = is_shp[is_ds]
ds_idx = cp.vstack(cp.where(is_ds)).T
cov, coh = emperical_co_sp(rslc,ds_idx,is_shp_ds)

In [27]:
cov.shape

(740397, 17, 17)

In [30]:
zarr.save_array('emperical_coh_ds.zarr',cp.asnumpy(coh),chunks=(100000, 17, 17),compressor=None)
zarr.save_array('emperical_cov_ds.zarr',cp.asnumpy(cov),chunks=(100000, 17, 17),compressor=None)